In [ ]:
!pip install git+https://github.com/toncenter/tvm_valuetypes.git

In [ ]:
#%load_ext autoreload
#%autoreload 2
from tvm_python import PyTVM, method_name_to_id, code_disasseble, pack_address, load_address
import codecs
from tvm_valuetypes import Cell, deserialize_boc
from dateutil.parser import parse 
import codecs

In [ ]:
def fix_op(item):
    return item.split('\t')[1].replace('execute ', '').replace('\n', '')

In [1]:
from tvm_python import parse_onchain_data

In [2]:
data = 'te6ccuECEgEAAbUAAAoAFABeAGgA8gD8AQYBEAFYAaAB6AIIAm4CiALQAxgDYgNqAQMAwAECASACAwFDv/CC62Y7V6ABkvSmrEZyiN8t/t252hvuKPZSHIvr0h8ewAQCASAFBgCGAGlwZnM6Ly9iYWZrcmVpYTNrdG14eG9idG42ZXhkNWp1ZWhmM2s0dG1jbDQ2Y2VqN2J1azNjZTJudWI3eGwyNTczaQIBIAcIAgEgDg8CAUgJCgFBv27U+UKnhIziywZrd6ESjGof+MQ/Q4otziRhK6n/q4sDDQFBvtUam/+G3nP3Ya609uHQxPc3i+wXmp0qn81UtlhfHnRMCwFBvu5eHQlqGfI5s5WmVQi17RtEg5HS4JLKbWVxemQgVehcDAAcAExQLW9VU0RDLVYxLjIAYgBFUUM2MUlRUmwwX2xhOTV0Mjd4aElwanhadDMydmwxUVFWRjJVZ1ROdXZEMThXLTQAFgBUT05TV0FQLUxQAUG/Ugje9G9aHU+dzmarMJ9KhRMF8Wb5Hvedkj71jjT5ogkQAUG/XQH6XjwGkBxFBGxrLdzqWvdk/qDu1yoQ1ATyMSzrJH0RAEYAVG9uU3dhcCBMaXF1aWRpdHkgUHJvdmlkZXIgSmV0dG9uLgAEADnaJaOB'

In [3]:
parse_onchain_data(data)

image
snake
name
snake
8DCBC3A12D433E473672B4CAA116BDA36890723A5C12594DACAE2F4C840ABD0B
snake
symbol
snake
description
snake
decimals
snake


{'image': 'snake',
 'name': 'snake',
 '8DCBC3A12D433E473672B4CAA116BDA36890723A5C12594DACAE2F4C840ABD0B': 'snake',
 'symbol': 'snake',
 'description': 'snake',
 'decimals': 'snake'}

# Just code

In [ ]:
PyTVM(code="te6ccgEBAQEAAwAAAnE=").run_vm()

### Stack is not reversed ;)

In [ ]:
SUB = "te6ccgEBAQEAAwAAAqE="
a = 3
b = 1

tvm = PyTVM(code=SUB)
tvm.set_stack([a, b])
tvm.run_vm()

#### Balance can be really big integer

In [ ]:
tvm = PyTVM(code=SUB)

tvm.set_c7(address="EQCI5yVsMccJDqrKH9knYAp0avK5fh-UBZ5Ns1xWOvIA2UKl", # some contracts may use MYADDR in code
           balanceGrams="9951186000", # or trying get own balance from c7
           randseed="1") # or maybe you use random in your contract :)

# Disassembler

In [ ]:
code_disasseble("te6ccgEBAQEAAwAAAnE=")

# Easy test examples

In [ ]:
# Simple PUSHINT 1
# You can define log_level if you want more stuff
tvm_result = PyTVM(code="te6ccgEBAQEAAwAAAnE=", log_level=4).run_vm()
assert tvm_result[0] == 1

print(f"Got 1 from TVM: {tvm_result[0]}")

In [ ]:
tvm_result

In [ ]:
# c3 is continuation, but we just serialize it to base64 BOC
# PUSHINT 1 PUSH c3
tvm_result = PyTVM(code="te6ccgEBAQEABQAABnHtQw==").run_vm()
tvm_result

In [ ]:
cell = Cell()
b64 = tvm_result[1]['value']
boc = codecs.decode(codecs.encode(b64, 'utf8'), 'base64')
deserialize_boc(boc)

In [ ]:
# c7 push (tuple)
tvm_result = PyTVM(code="te6ccgEBAQEABAAABO1H").run_vm()
tvm_result[0] # just default c7

## Change c7

In [ ]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H")

In [ ]:
tvm.set_c7(translt="100",
           unixtime=179, 
           globalConfig="te6ccgEBAQEABAAABO1H",
           randseed="100", 
           address="EQBq5JLG8-1juc-N95IFBnOM2NGrcJsMoFYFoWH-DaVJ2lVw",
           balanceGrams=str(100000000))

In [ ]:
tvm.run_vm()

## Get detailed result

In [ ]:
print("Exit code: ", tvm.exit_code,
"\nC5: ", tvm.actions, 
"\nGas credit: ",tvm.gas_credit, 
"\nGas used: ",tvm.gas_used, 
"\nC4: ",tvm.new_data, 
"\nVM Steps: ",tvm.vm_steps,
"\nVM final hash: ",tvm.vm_final_state_hash,
"\nVM init hash: ",tvm.vm_init_state_hash,
"\nVM success: ",tvm.success)

## Set stack

In [ ]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H", log_level=6)

In [ ]:
# actually python3 & TON have really big integers :))
tvm.set_stack([
    190009329293929392939239293293929329392949949949299008888888880999999888, 
    {'type': "cell",
     'value': 'te6ccgEBAQEABAAABO1H'}, 
    [{'type': "cellSlice",
     'value': 'te6ccgEBAQEABAAABO1H'},
     12312312313213132,
     [1231232132, 1321231231, 57, 179]
    ]])

In [ ]:
tvm.run_vm()

## C4 (Data) usage example

In [ ]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=2)

In [ ]:
# You can see, that 179 was loaded from c4, and cellSlice on stack
tvm.run_vm()

## Gas Limit 

In [ ]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz")

In [ ]:
print(tvm.gas_used, tvm.gas_credit)
tvm.set_gasLimit(26) # You can also pass max gas set_gasLimit(gas_limit=26, gas_max=26)
print(tvm.gas_used, tvm.gas_credit)
tvm.run_vm() # out of gas

In [ ]:
tvm.exit_code, tvm.gas_used

## Libs 

This is just load example, they will pass to VM, but I'm little bussy to get code of loading libs, check out [akifoq](https://github.com/akifoq) github

In [ ]:
tvm.set_libs([{'type': 'cell', 'value': 'te6ccgEBAQEABwAACu1E0NM/'}])

## Stack dump log Level

In [ ]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=4)

In [ ]:
tvm.run_vm()

## Allow debug & same c3

In [ ]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=4,
            allow_debug=True,
            same_c3=False)

In [ ]:
tvm.run_vm()

## Accout get methods

In [ ]:
# get account code & data from dton.io
tvm = PyTVM(code="te6ccuECDAEAAikAABoAJAAuADYAQACWAZYClAKuAv4DzARSART/APSkE/S88sgLAQIBIAIDAgFIBAUABPIwAgLNBgcAUaA4WdqJoaYBpj/0gfSB9IH0AahhofSB9AH0gfQAYQQgjJKwoBWAAqsBAvfQDoaYGAuNhJL4JwfSAYdqJoaYBpj/0gfSB9IH0AahgTYAD5aMoRa6ThAVnHIBkcHCmg44LJL4RwKKJjgvlw+gJpj8EIAonGyIldeXD66Z+Y/SAYICsDZGWACuWPqAHniwDniwDniwD9AWZk9qpwGxPjgHGBA+mP6Z+YEMCAkB92YIQO5rKAFJgoFIwvvLhwiTQ+kD6APpA+gAwU5KhIaFQh6EWoFKQcIAQyMsFUAPPFgH6AstqyXH7ACXCACXXScICsI4XUEVwgBDIywVQA88WAfoCy2rJcfsAECOSNDTiWnCAEMjLBVADzxYB+gLLaslx+wBwIIIQX8w9FILABY3EDhHZRRDMHDwBQFKwAGSXwvgIcACnzEQSRA4R2AQJRAkECPwBeA6wAPjAl8JhA/y8AoAyoIQO5rKABi+8uHJU0bHBVFSxwUVsfLhynAgghBfzD0UIYAQyMsFKM8WIfoCy2rLHxnLPyfPFifPFhjKACf6AhfKAMmAQPsAcQZQREUVBsjLABXLH1ADzxYBzxYBzxYB+gLMye1UAIIhgBjIywUqzxYh+gLLassfE8s/I88WUAPPFsoAIfoCygDJgwb7AHFVUAbIywAVyx9QA88WAc8WAc8WAfoCzMntVHEzQWc=", 
            data="te6ccuECAgEAAKEAAKIBQgGbMateFUAFhO5hst/wg3EW0Py1B42TlkvL6cBf1qFBsb/KXWpD4YgAdP2guq8i0ybWx7E2zTGM2WdwO9WauK3yzhASGFCjzN4wSMCWvrAEAQCbgBRyaww+87XrNCetowXCyAQhgF8xx74x+06XHrVig1fjCujOry8BAANKN8ZnM0OzYOENTkOEg7VVgFog5fBWdCtqQro1MRmU1dGdXl4CusI7qA==", 
            log_level=4)

In [ ]:
tvm.set_c7(address="EQCI5yVsMccJDqrKH9knYAp0avK5fh-UBZ5Ns1xWOvIA2UKl", # some contracts may use MYADDR in code
           balanceGrams=str(100000000), # or trying get own balance from c7
           randseed="1") # or maybe you use random in your contract :)

In [ ]:
method_name_to_id("get_sale_data") # get methods have own numbers :)

In [ ]:
tvm.set_stack([method_name_to_id("get_sale_data")]) # long story short, you need to pass method_id to stack

In [ ]:
tvm.run_vm() # this is seqno of this wallet :)

## Method names

In [ ]:
smart_methods = {}

for name in ["dnsresolve", "get_jetton_data", "get_wallet_address", "get_collection_data",
             "get_nft_address_by_index", "get_nft_data", "get_royalty_params", "hello_world",
             "get_owner", "seqno", "get_public_key", "get_version",
             "get_marketplace_address", "get_sale_data", "get_plugin_list", "is_plugin_installed",
             "get_subwallet_id", "get_editor", "get_is_closed", 
             'get_reveal_mode', 'get_reveal_data', 
             
             'get_full_domain', 'get_username', 'get_auction_info', 
             'get_telemint_token_name', 'get_telemint_auction_state', 
             'get_telemint_auction_config', 'get_offer_data',
            ]:

    smart_methods[name] = method_name_to_id(name)

In [ ]:
smart_methods

# Run transaction & stack step dump

In [ ]:
transaction = {
        "account_state_state_init_code": "te6ccuEBAQEAYgDEAMD/ACDdIIIBTJe6lzDtRNDXCx/gpPJggwjXGCDTH9Mf0x/4IxO78mPtRNDTH9Mf0//RUTK68qFRRLryogT5AVQQVfkQ8qP4AJMg10qW0wfUAvsA6NEBpMjLH8sfy//J7VRcWvJx",
        "account_state_state_init_data": "te6ccuEBAQEAKgBUAFAAABfFKamjF1EWudQc3BYnnT5cgwKbbuOMnLoj2nddsBRI8MyLn7VbLd5twQ==",
        "gen_utime": "2022-10-29 23:17:14",
        "account_storage_balance_grams": 3503394350113822,
        "in_msg_cell": "te6ccuECAgEAALgAAOYBcAHfiAELXvHRoGvSQCL5tMqMK5vm46jIkBbFPCIaeKYhsLVmxAUv1tIPUGD5rk63+HQIldqZpKMW5papR2c6Je1YCY71+7WGOJg9/+Tmmk+6PG/8ACrE9tk+bE8f6fVUpmDc3uBZTU0YuxrwtygAAL4oHAEAhkIAQGoJHCCLPlTE1kiSIeZM0lzg1DrCKbIT/7fsLkkcrXSovBvI3AAAAAAAAAAAAAAAAAAAAAAAADE5MjY1MjUwNjX5zgaJ",
        "in_msg_body": "te6ccuECAgEAAJUAAKABKgGapfraQeoMHzXJ1v8OgRK7UzSUYtzS1Sjs50S9qwEx3r92sMcTB7/8nNNJ90eN/4AFWJ7bJ82J4/0+qpTMG5vcCympoxdjXhblAAAXxQMBAIZCAEBqCRwgiz5UxNZIkiHmTNJc4NQ6wimyE/+37C5JHK10qLwbyNwAAAAAAAAAAAAAAAAAAAAAAAAxOTI2NTI1MDY1lbIkmg==",
        "in_msg_value_grams": None,
        "in_msg_type": "ext_in_msg_info",
        "in_msg_dest_addr_address_hex": "85AF78E8D035E920117CDA654615CDF371D464480B629E110D3C5310D85AB362",
        "in_msg_dest_addr_workchain_id": 0,
        "lt": 32426921000001,
        "in_msg_created_lt": None
      }


transaction['in_msg_cell'] = {'type': 'cell', 'value': transaction['in_msg_cell']}
transaction['in_msg_body'] = {'type': 'cellSlice', 'value': transaction['in_msg_body']}

tvm = PyTVM(code=transaction['account_state_state_init_code'], 
            data=transaction['account_state_state_init_data'])

tvm.set_c7(unixtime=int(parse(transaction['gen_utime']).timestamp()), 
           translt=str(int(transaction['lt'])), 
           balanceGrams=str(int(transaction['account_storage_balance_grams'])),
           address=str(transaction['in_msg_dest_addr_workchain_id']) + ':' + transaction['in_msg_dest_addr_address_hex'])

tvm.set_stack([
    str(int(transaction['account_storage_balance_grams'])),
    str(int(transaction['in_msg_value_grams'] if transaction['in_msg_value_grams'] else 0)),
    transaction['in_msg_cell'],
    transaction['in_msg_body'],
    -1 if 'ext' in transaction['in_msg_type'] else 0
])

tvm.run_vm()


In [ ]:
list(map(fix_op, tvm.get_ops()))

In [ ]:
tvm.get_stacks()

## Calculate jetton wallet address

In [ ]:
tvm = PyTVM(code="te6ccuECCwEAAeoAABoAJAAuADoBLgHGAsgC/AMuA7AD1AEU/wD0pBP0vPLICwECAWICAwICzAQFAgN6YAkKA+nZkQ44BIrfBoaYGAuNhIrfB9IBgA6Y/pn/aiaH0AfSBqahgTYArHGBqaqKrjgvlwJID9IH0AahgQaEAwa5D9ABgSiBooIXgG0CqQZCgCfQEsZ4tmZmT2qnATQQg97svvXXGBGpLgAfGBGAJgAnGBL4LCB/l4QGBwgAk7vwUIgG4KhAJqgoB5CgCfQEsZ4sA54tmZJFkZYCJegB6AGWAZJB8gDg6ZGWBZQPl/+ToO8AMZGWCrGeLKAJ9AQnltYlmZmS4/YBAP42A/oA+kD4KFQSCHBUIBNUFAPIUAT6AljPFgHPFszJIsjLARL0APQAywDJ+QBwdMjLAsoHy//J0FAIxwXy4EoSoQNQJMhQBPoCWM8WzMzJ7VQB+kAwINcLAcMAjh+CENUydttwgBDIywVQA88WIvoCEstqyx/LP8mAQvsAkVviADA1FccF8uBJ+kAwWchQBPoCWM8WzMzJ7VQALlFDxwXy4EnUMAHIUAT6AljPFszMye1UAH2tvPaiaH0AfSBqahg2GPwUALgqEAmqCgHkKAJ9ASxniwDni2ZkkWRlgIl6AHoAZYBk/IA4OmRlgWUD5f/k6EAAH68W9qJofQB9IGpqGD+qkEDmPMC3", 
            data="te6ccuECEwEAA40AAFYA8gEMARYBIAFAAUoB0AHaAeQCoAK2A64DuARWBVQGLAagBxoCTVGZyCzACADv1+I+xGKwYr5T+5ikwv1q2BoKzwD1yyab0X4Qax9zAQECAJgBaHR0cHM6Ly9yYXcuZ2l0aHVidXNlcmNvbnRlbnQuY29tL2Rpc2ludGFyL29uZXRpbWVuZnQvbWFpbi9mYWtlX2pldHRvbi5qc29uART/APSkE/S88sgLAwIBYgQFAgLLBgcAG6D2BdqJofQB9IH0gahhAgEgCAkAgdIAg1yHtRND6APpA+kDUMATTHyGCEBeNRRm6AoIQe92X3roSsfLixdM/MfoAMBOgUCPIUAT6AljPFgHPFszJ7VSAgEgCgsCAVgMDQC30QY4BJL4JwAOhpgYC42EqJr4H4CHB9IH0gGP0AGLjrkP0AGP0AGAFpj5DBCAfFP1LdSpiaLPgG8BDBCAvGoozdSxiiIgH4B3AawQgsr4PeXUms+AfwL4JCB/l4QAEWvpEMMAA8uFNgHxUD0z/6APpAIfAG7UTQ+gD6QPpA1DBRNqFSKscF8uLBKML/8uLCVDRCcFQgE1QUA8hQBPoCWM8WAc8WzMkiyMsBEvQA9ADLAMkg+QBwdMjLAsoHy//J0AT6QPQEMfoAINdJwgDy4sR3gBjIywVQCM8WcPoCF8trE8yA4CASAPEACaghAXjUUZyMsfGcs/UAf6AiLPFlAGzxYl+gJQA88WyVAFzCORcpFx4lAIqBOggXUwoBS88uLFBMmAQPsAECPIUAT6AljPFgHPFszJ7VQC9TtRND6APpA+kDUMAjTP/oAUVGgBfpA+kBTW8cFVHNtcFQgE1QUA8hQBPoCWM8WAc8WzMkiyMsBEvQA9ADLAMn5AHB0yMsCygfL/8nQUA3HBRyx8uLDCvoAUaihgScQZrYIoYEnEKAYoSeXEEkQODdfBOMNJdcLAcMAI4BESANM7UTQ+gD6QPpA1DAH0z/6APpAMFFRoVJJxwXy4sEnwv/y4sIFgU4goBa88uLDghB73ZfeyMsfFcs/UAP6AiLPFgHPFslxgBjIywUkzxZw+gLLaszJgED7AEATyFAE+gJYzxYBzxbMye1UgAHBSeaAYoYIQc2LQnMjLH1Iwyz9Y+gJQB88WUAfPFslxgBDIywUkzxZQBvoCFctqFMzJcfsAECQQIwB2wgCwjiGCENUydttwgBDIywVQCM8WUAT6AhbLahLLHxLLP8ly+wCTNWwh4gPIUAT6AljPFgHPFszJ7VQdowYo")

tvm.set_c7(address="EQBK3b2y8Ogu8l0pBuU3V17Q2Ln2esTNHFsWMuyOUTdZkbH4")

tvm.set_stack([
    pack_address("EQB36_EfYjFYMV8p_cxSYX61bA0FZ4B65ZNN6L8INY-5gL6w"),
    method_name_to_id('get_wallet_address')
])

load_address(tvm.run_vm()[0]['value'])

## Test

In [ ]:
code = 'te6ccuECQQEACYEAABoAaAByAHwAhgCQAJoApACuAPgB6gI2An4CmgKkAq4CuALCAswC1gMsA3IDzAQYBCIELAT8BfoGAAb2B/QImAioCLIIvAjGCNAJJAl+CbAJwgnMCdYKlAtoC6YMhA0WDg4OIg4sDjYOQA5KD0wPhhA6ELAQ0hDwEPoRRhI8EuYTAgEU/wD0pBP0vPLICwESAfYj51zhqnziR3CmpH+GXrtDvPQG9Y9MaUlZVl6oO9ewAAhiAgMCAssEBQIBIDIzAgEgBgcCASAODwIBIAgJAgEgDA0CASAKCwBF9BCHhPyMa4EEAIZGWCqANniygCfQEKZbUJZY+JZY/kgP2AQA7UAdDTAwFxsJJfA+D6QDDwECHXScEgk/LATt4B0x8hghBpPTlQuo4sMWwS0z8w+EfQghCoywCtcIAQyMsFUAXPFiT6AhTLahPLH8s/Ac8WyYBA+wCOICGCEElm6Ji6lFsB8B+OEPhBE8cFk1nwHpVfA/LATuLi4oAEdHAgcY4UA3qpDKYwJKgSoAOqBwKkIcAARDDmMGwSAaoCEs8BgAQ/P7/IgMcLqZH6PjfSkTdZzQFqGCoQAhN6C4FImXF0Ca+BwAF2XTIywISygfL/8nQgIBIBARAgEgISICASASEwIBIBgZAgEgFBUCASAWFwBRO1E0PpAAfhh+kAB+GjTPwH4YtIAAfhj1AH4ZNQB+GXUAfhm1AH4Z9GAAQT4R/hG+EX4RPhD+ELI+EHPFvhIzxbLP8oAzMzMzMntVIABVPhE0PpAAfhp+kAB+Gr6AAH4a/oAAfhs+gAB+G3THwH4btMfAfhv1DD4cIABHPhQ+E/4Tsj4Sc8W+ErPFvhL+gL4TPoC+E36Assfyx/MyfhkgAgEgGhsCASAeHwDLG8jyI0EDxwYXRoIGZpbGw9InJnYiiDPFlAD8AKLEsjPFgHwAosSyM8WAfACi3KSIgZD0iTYzxZQBPACixIIzxZY8AKLIgaIzxYh8AKLF2jPFljwAosmgtjPFgHwAotHoiLz6M8WgAvUyI0GGRhdGE6aW1hZ2Uvc3ZnK3htbDt1dGY4LIM8Wi9PHN2ZyB2aWV3Qm94PYzxaLEijPFotDAgMCCM8WI/ACixIIzxYi8AKLEijPFotyB3aWR0aD2M8WixIozxZQA/ACixIozxaLggaGVpZ2h0PYzxaLEijPFgHwAomAcHQACIgDyzxbJcFiAIPQXyItyB4bWxucz2M8WixIozxaNBpodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2Z4M8WixIozxaNBEgc2hhcGUtcmVuZGVyaW5nPYM8WixIozxaLpjcmlzcEVkZ2VzjPFosSKM8WixPozxbJcViAIPQXcgH3APTB9MHMdMHURSoAdMHURWoAdMHbwACntMH0wfTB1UgbwMSb4wB5HBSAtMPAY5MIMcBwP+T1DDQ3tMH0wchwwCOJVM6qFKAoFNbqFKAoFNMqASlUoBvgUEwVCTQ8BTJVCC6gCD0FwiRMeJZoAmkU5a6ljlwAqRAGd5QmYCAAnwB0w/TD9MP0w/0BDBtUUKoUTKoQwTwFXAFjiBTQ4AQ9A5voTDTDzEG0w8H9AQwgBD0atBUQzDwFgSkBORsQsiLY8L3N2Zz6M8WyQKAIPQXgAAzkEHhfCAECASAjJAIBICkqAgEgJSYCASAnKABPNMHINMH0wcwbVEkqFEUqFjwFRAj8BbIi2PC9zdmc+jPFskCgCD0F4ABVPgl+BXTD4Aw1yH0BDDIcAOOFFMhgBD0Dm+hMNMPMPgRAcsPAqQC5GwhyYAAtAHIyz/4KM8WyXAgyMsBE/QA9ADLAMmAADRwMvkA8A2ACASArLAIBIC4vALk+CP4ToIIgOvoyMsf+EHPFvhL+gL4TPoC+E36AssfUAXPFhTLH8n4I/hPoHDIywBQBM8WWPoCE8sfyx/4Ss8W+CjPFszJ+FABcCDIywET9AD0AMsAyXAh+QDwDQGABzT4RdDUMdQx1DDQ8Bn4QvhG8BpwIfAbcMjLAcnQcFMCVSDwHIIQBfXhAPgjyCTPFss/FszJEDReMXF3gBjIywVQBc8WUAP6AhPLaxPMzMkB+wCCEAX14QCCEAgi2K7Iyx/JIhA0WXGAtADp3gBjIywVQBc8WUAP6AhPLaxPMzMkB+wD4QqT4YgHXDF/IoIKPNUsup0y0gAB+GP6QNH4aPARjsgighBhvd+LupUy1NH7BI63IoIQ2/r4F7qZMtTU0QH7BO1UjqIighBZ2iAZupNfA3COkyKCENTXHpm6mDLUAfhl0fAR4w7i4uLil/hBcIBA8AfegMACNPAS+ElSIMcF+EETxwUSsfLgToIQEeGjAL7y4Ev4Q/LgSvAd+GlwdPsC+EhwgwZwgBDIywVQBM8WWPoCEstqyQH7APAT8BGAB8iKCEO1zye26jm0ighAte7CGupwy8BLUAfhw0fAT8BGOVSKCECledam6mTL6QAH4YdHwEY5AIoIQpfBz57qYMtQB+GfR8BGOLDEBghAQ1gp2uo4cMHD4QXCBAKBwgBDIywVQBM8WWPoCEstqyQH7AJPywE7i4uLi4w0xABAy1AH4ZNHwEQIBIDQ1AgEgPD0CAUg2NwIBIDo7Avmy13wEPhF0NQx1DHUMNAB0PAXcMjLB4uk1lcmlkaWFuICOM8WWPACyXDIyweJzxbJgvDJBG96N60Op87nM1WYT6VCiYL4s3yPe87JH3rHGnzRBILwgqNTf/Dbzn7sNdae3DoYnubxfYLzU6VT+aqWywvjzokEccjLB/QAyYDg5ADWwB3wEPAS+EP4QvhB+Ej4SPhJ+Er4S/hM+E2AAsE1lcmlkaWFuIGlzIHRoZSBmaXJzdCBmdWxseSBvbi1jaGFpbiBjb2xsZWN0aW9uIG9uIFRPTiBibG9jay1jaGFpbiBvZiByYW5kb20gY2hhcmFjdGVycy4AcoLwYQXWzHavQAMl6U1YjOURvlv9u3O0N9xR7KQ5F9ekPj1tgwf0F14hgwf0FxKDB/QXcMjLB/QAyQAdtdr+Ah8I+hph+mH/SAYQABm09H4CHwjeA04APgNwAgEgPj8AR7hr2NB5odHRwczovL3QubWUvK1JETG1icHRpZHZ4aVpUWXmCAHvtgt+Ah8IuhqamoY6hgBaHgMAOh4DDjkZYP6AGTBeC98XSzONLlGNMamblECBwkNZZVU22pRZFeBboXInMZzgTjkZYP6AGTBeDCC62Y7V6ABkvSmrEZyiN8t/t252hvuKPZSHIvr0h8etsGD+guJQYP6C8GDrPgFwQAClt3oxoMN7kzlzo3txc0tzo5N7m4MrG6NLe3FzsYQRoMt7kzlzo3txc3MzoXMbe2NjKxujS3txc7GEEaCze5M5c6N7cXNzM6Fzk3vLC2OjyXOxhBAAGHDIywf0AMn4QvhBEmNjRgc='
data = '...'

In [ ]:
len(data)

In [ ]:
tvm = PyTVM(code=code, data=data)
tvm.set_stack([method_name_to_id("get_collection_data")])
tvm.set_c7(address="0:151A193FDEB500DF2A59019D5291519921A0EDED476FC00E23B12191291A3E31", balanceGrams="24507456")

In [ ]:
tvm.run_vm()